In [1]:
import pandas as pd
import hopsworks
import CleanInputFunction
import TopicExtractionFunction

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\steem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\steem\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\steem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Connect to Hopsworks
project = hopsworks.login()
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5270


Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Read in training data
data = pd.read_csv('data/RoundedLabelled.csv')
data = data.rename(columns={'title': 'title', 'topic': 'subjects', 'stance / attitude towards': 'stance', 'article URL': 'url', 'savedAt': 'saved_at', 'main topic': 'labelled_topic'})
data = data.drop(['query', 'language', 'Clickbaityness/credibility (0 is good, 1 is very bad)', 'labelled_topic'], axis=1) # labelled_topic is dropped as this is only for testing the topic extraction and is not used as training data, Hopsworks requires all columns to be filled which won't always be the case for labelled_topic
data = data[data['stance'].notnull()]
data['stance'] = data['stance'].astype('double')
data

,title,saved_at,url,stance
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0
...,...,...,...,...
567,Ohio county reports more than 80 measles cases...,31/12/2022 17:13,https://www.foxnews.com/health/ohio-county-rep...,1.0
568,Idaho college killings suspect is criminology ...,31/12/2022 17:13,https://news.yahoo.com/suspect-deaths-idaho-st...,2.0
569,What do your favorite James Webb images say ab...,31/12/2022 17:13,https://www.dailymail.co.uk/sciencetech/articl...,2.0
570,"Mega Millions drawing produces no winner, jack...",31/12/2022 17:13,https://www.cnn.com/2022/12/31/us/mega-million...,1.0


In [4]:
# Pre-process
data = CleanInputFunction.clean_input(data)
display(data)

,title,saved_at,url,stance,title_stance,title_topic
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...
...,...,...,...,...,...,...
567,Ohio county reports more than 80 measles cases...,31/12/2022 17:13,https://www.foxnews.com/health/ohio-county-rep...,1.0,ohio county reports more than 80 measles cases...,ohio county reports 80 measles cases majority ...
568,Idaho college killings suspect is criminology ...,31/12/2022 17:13,https://news.yahoo.com/suspect-deaths-idaho-st...,2.0,idaho college killings suspect is criminology ...,idaho college killings suspect criminology phd...
569,What do your favorite James Webb images say ab...,31/12/2022 17:13,https://www.dailymail.co.uk/sciencetech/articl...,2.0,what do your favorite james webb images say ab...,favorite james webb images say daily mail
570,"Mega Millions drawing produces no winner, jack...",31/12/2022 17:13,https://www.cnn.com/2022/12/31/us/mega-million...,1.0,mega millions drawing produces no winner jackp...,mega millions drawing produces winner jackpot ...


In [5]:
# Add topics
data, _ = TopicExtractionFunction.extract_topics(data)
display(data)

2022-12-31 17:44:16,797 INFO: Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2022-12-31 17:44:19,833 INFO: Use pytorch device: cpu


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2022-12-31 17:44:30,203 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2022-12-31 17:44:30,423 INFO: Use pytorch device: cpu


,title,saved_at,url,stance,title_stance,title_topic,predicted_topic
0,Fed interest rate decision today: Live updates...,14/12/2022 18:11,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...,"inflation, fed, rates"
1,Moscow police should hand over control of Idah...,14/12/2022 18:11,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...,"idaho, coroner, arrested"
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022 18:11,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...,"africa, cup, al"
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022 18:11,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...,"nintendo, gematsu, switch"
4,Mortgage demand inches higher as interest rate...,14/12/2022 18:11,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...,"cnbc, fed, cramer"
...,...,...,...,...,...,...,...
567,Ohio county reports more than 80 measles cases...,31/12/2022 17:13,https://www.foxnews.com/health/ohio-county-rep...,1.0,ohio county reports more than 80 measles cases...,ohio county reports 80 measles cases majority ...,"strep, children, died"
568,Idaho college killings suspect is criminology ...,31/12/2022 17:13,https://news.yahoo.com/suspect-deaths-idaho-st...,2.0,idaho college killings suspect is criminology ...,idaho college killings suspect criminology phd...,"idaho, coroner, arrested"
569,What do your favorite James Webb images say ab...,31/12/2022 17:13,https://www.dailymail.co.uk/sciencetech/articl...,2.0,what do your favorite james webb images say ab...,favorite james webb images say daily mail,"webb, telescope, james"
570,"Mega Millions drawing produces no winner, jack...",31/12/2022 17:13,https://www.cnn.com/2022/12/31/us/mega-million...,1.0,mega millions drawing produces no winner jackp...,mega millions drawing produces winner jackpot ...,"meghan, harry, netflix"


In [6]:
# Save result to feature_store
article_cleaned_feature_store = feature_store.get_or_create_feature_group(
    name="training_data_stance",
    version=1,
    primary_key=["url"],
    description="Articles with predicted topic")
article_cleaned_feature_store.insert(data, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/572 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5270/jobs/named/training_data_stance_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x23f78032670>, None)